# Notebook 06: Advanced Concepts

## Content
- object-oriented programming (OOP)
- the `lambda` command (anonymous functions)
- iterators and generators

## Remember jupyter notebooks
- To run the currently highlighted cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>.
- To get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## Object-oriented programming
A `class` is a blueprint for a container like data structure containing variables (**attributes**) and functions (**methods**). An `object` is an actual instance of a `class`.

**Example**: let's build a rotation matrix, i.e.,

$$\mathbf{R}(\phi) = \begin{pmatrix} \cos(\phi) & -\sin(\phi) \\ \sin(\phi) & \cos(\phi) \end{pmatrix},$$

which rotates a vector $\mathbf{r}\in\mathbb{R}^2$ by the angle $\phi$.
As a function, it could look like this:

In [ ]:
def rotate(vector, angle):
    """Rotate the given vector by the given angle"""
    rad = np.pi * angle / 180
    rotation = np.array([
        [np.cos(rad), -np.sin(rad)],
        [np.sin(rad), np.cos(rad)]])
    return np.dot(rotation, np.asarray(vector))


print(rotate([1, 0], 90))

Now, let's build a class to encapsulate this behaviour!


In [ ]:
class RotationMatrix(object):
    """This will rotate a 2D vector by some angle phi"""
    
    def __init__(self, angle):
        """
        Arguments:
            angle (float): angle in °
        """
        self.angle = angle % 360


rotation = RotationMatrix(90)
print(rotation.angle)
print(rotation)

Printing `rotation` tells us that it is an **object** as well as its address — but nothing more. To learn about its actual `angle`, we need to look up the `angle` **attribute**.

In [ ]:
class RotationMatrix(object):
    """This will rotate a 2D vector by some angle phi"""
    
    def __init__(self, angle):
        """
        Arguments:
            angle (float): angle in °
        """
        self.angle = angle % 360

    def __repr__(self):
        """Represent the object by some informative string"""
        return f'Rotation by {self.angle:.2f}°'

    
rotation = RotationMatrix(90)
print(rotation)

The `__repr__` method is one of many **magic methods** in Python. These methods have two leading and trailing underscores and are called in very specific circumstances, e.g., when we **initialise** the object (`__init__`) or if we attempt to **print** the object (`__repr__`).

In [ ]:
rotation2 = RotationMatrix(-90)
print(rotation2)

Much better! But remember that our class should represent a **matrix**, and matrices can be multiplied...

In [ ]:
rotation * rotation2

We want to make matrix multiplication available. For that, we need another **magic method**, `__mul__`:

In [ ]:
class RotationMatrix(object):
    """This will rotate a 2D vector by some angle phi"""
    
    def __init__(self, angle):
        """
        Arguments:
            angle (float): angle in °
        """
        self.angle = angle % 360

    def __repr__(self):
        """Represent the object by some informative string"""
        return f'Rotation by {self.angle:.2f}°'
    
    def __mul__(self, other):
        """Multiply two rotations by adding their angles"""
        return RotationMatrix(self.angle + other.angle)


rotation = RotationMatrix(90)
print(rotation)

rotation2 = RotationMatrix(-90)
print(rotation2)

print(rotation * rotation2)
print(rotation.__mul__(rotation2))

Here, we have used that a product of two rotation matrices is another rotation matrix where the angle is the sum of the two original angles.

Note that we now have (rotation) matrix multiplication available without even having implemented the actual rotation of a vector...

In [ ]:
class RotationMatrix(object):
    """This will rotate a 2D vector by some angle phi"""
    
    def __init__(self, angle):
        """
        Arguments:
            angle (float): angle in °
        """
        self.angle = angle % 360
        self.rad = np.pi * self.angle / 180
        self.rotation_matrix = np.array([
            [np.cos(self.rad), -np.sin(self.rad)],
            [np.sin(self.rad), np.cos(self.rad)]])
        
    def __repr__(self):
        """Represent the object by some informative string"""
        return f'Rotation by {self.angle:.2f}°'
    
    def __mul__(self, other):
        """Multiply two rotations by adding their angles"""
        return RotationMatrix(self.angle + other.angle)

    def rotate(self, vector):
        """Rotate the vector"""
        return np.dot(self.rotation_matrix, np.asarray(vector))


rotation = RotationMatrix(90)
print(rotation.rotate([1, 0]))

And now, let's create a new class which **inherits** from `RotationMatrix`:

In [ ]:
class RotationMatrix2(RotationMatrix):
    """This is an example on inheritance"""

    def __mul__(self, other):
        """Multiply two rotations by adding their angles"""
        return RotationMatrix2(self.angle + other.angle)

    def __call__(self, vector):
        """Rotate the vector"""
        return self.rotate(vector)


r1 = RotationMatrix2(180)
r2 = RotationMatrix(-90)

print(isinstance(r1, RotationMatrix), isinstance(r1, RotationMatrix2))
print(isinstance(r2, RotationMatrix), isinstance(r2, RotationMatrix2))

In [ ]:
print(isinstance(r1 * r2, RotationMatrix), isinstance(r1 * r2, RotationMatrix2))
print(isinstance(r2 * r1, RotationMatrix), isinstance(r2 * r1, RotationMatrix2))

In [ ]:
print(r1 * r2)
print((r1 * r2)([1, 0]))

**Example**: Some of Python's **magic methods** and their corresponding binary/unary operators:

In [ ]:
class Tester(object):
    def __add__(self, value):
        print(f'__add__({value})')
    def __sub__(self, value):
        print(f'__sub__({value})')
    def __mul__(self, value):
        print(f'__mul__({value})')
    def __truediv__(self, value):
        print(f'__truediv__({value})')
    def __floordiv__(self, value):
        print(f'__floordiv__({value})')
    def __mod__(self, value):
        print(f'__mod__({value})')
    def __pow__(self, value):
        print(f'__pow__({value})')
    def __and__(self, value):
        print(f'__and__({value})')
    def __or__(self, value):
        print(f'__or__({value})')
    def __xor__(self, value):
        print(f'__xor__({value})')
    def __eq__(self, value):
        print(f'__eq__({value})')
    def __neq__(self, value):
        print(f'__neq__({value})')
    def __gt__(self, value):
        print(f'__gt__({value})')
    def __ge__(self, value):
        print(f'__ge__({value})')
    def __lt__(self, value):
        print(f'__lt__({value})')
    def __le__(self, value):
        print(f'__le__({value})')
    def __radd__(self, value):
        print(f'__radd__({value})')
    def __iadd__(self, value):
        print(f'__iadd__({value})')


t = Tester()
t + 1
t - 1
t * 1
t / 1
t // 1
t % 1
t ** 1
t & 1
t | 1
t ^ 1
t == 1
t != 1
t > 1
t >= 1
t < 1
t <= 1
1 + t
t += 1

## The `lambda ` command
Python provides a mechanism to make anonymous functions. But fiRst, let's remember how to use a references to functions:

In [ ]:
def func(x):
    print('func here: ', x)


func([0, 1, 2])

In [ ]:
func_alias = func

func_alias([3, 4, 5])

We can create variables which are pointing to functions. With `lambda` we can create a function without a name and let some variable point to it:

In [ ]:
anonymous_func = lambda x: print(f'lambda here: {x}')

anonymous_func([6, 7, 8])

The pattern is like this:

```Python
lambda variable: transform(variable)
```

Let's combine this with OOP and create a class for composing mathematical functions:

In [ ]:
class Function():
    """This examples leverages lambda and OOP"""

    def __init__(self, expression):
        """
        Arguments:
            expression (function): some mathematical transformation
        """
        self.expression = expression

        
f = Function(lambda x: x**2)

x = np.linspace(-2, 2, 100)
plt.plot(x, f.expression(x))

We now use yet another of Python's **magic methods**, `__call__`, which makes an object **callable** like a function:

In [ ]:
class Function(object):
    """This examples leverages lambda and OOP"""

    def __init__(self, expression):
        """
        Arguments:
            expression (function): some mathematical transformation
        """
        self.expression = expression

    def __call__(self, x):
        """Apply transformation to the given data

        Arguments:
            x (float or numpy.ndarray): some data to transform
        """
        return self.expression(x)


f = Function(lambda x: x**2)
plt.plot(x, f(x))

In [ ]:
class Function(object):
    """This examples leverages lambda and OOP"""

    def __init__(self, expression):
        """
        Arguments:
            expression (function): some mathematical transformation
        """
        self.expression = expression

    def __call__(self, x):
        """Apply transformation to the given data

        Arguments:
            x (float or numpy.ndarray): some data to transform
        """
        return self.expression(x)

    def __or__(self, other):
        """Compose a new Function

        Arguments:
            other (Function): other wraps self
        """
        return Function(lambda x: other(self(x)))


f = Function(lambda x: x**2)
g = Function(lambda x: np.exp(-x))

h = f | g

plt.plot(x, h(x))

In [ ]:
class Function(object):
    """This examples leverages lambda and OOP"""

    def __init__(self, expression):
        """
        Arguments:
            expression (function): some mathematical transformation
        """
        self.expression = expression

    def __call__(self, x):
        """Apply transformation to the given data

        Arguments:
            x (float or numpy.ndarray): some data to transform
        """
        return self.expression(x)

    def __or__(self, other):
        """Compose a new Function

        Arguments:
            other (Function): other wraps self
        """
        return Function(lambda x: other(self(x)))

    def derivative(self, x, dx=0.01):
        """Differentiate numerically

        Arguments:
            x (float or numpy.ndarray): where to differentiate self
            dx (float or numpy.ndarray): differentiation step size
        """
        dy = self(x + 0.5 * dx) - self(x - 0.5 * dx)
        return dy / dx


f = Function(lambda x: x**2)
g = Function(lambda x: np.exp(-x))

h = f | g

plt.plot(x, h(x), label='$g \circ f$')
plt.plot(x, h.derivative(x), label='$(g \circ f)^\prime$')
plt.legend(fontsize=15)

## Iterators and generators
We revisit the Fibonacci problem:

$$f_i = f_{i-1} + f_{i-2},\quad i\geq2,\, f_0=f_1=1$$

In the second notebook, we solved this problem with the code snippet

```Python
a, b = 1, 1
while True:
    a, b = a + b, a
    if a < 100:
        print(a)
    else:
        break
```

Now, we create an iterator which we could use in a for loop:

In [ ]:
class Fibonacci(object):
    """An iterator over Fibonacci numbers"""
    
    def __init__(self, stop):
        """Arguments:
            stop (int): endpoint for the iteration
        """
        self.stop = stop
        self.a, self.b = 1, 1
        
    def __iter__(self):
        """This is what make this class an iterator"""
        return self
    
    def __next__(self):
        """This is what actually computes the next number"""
        self.a, self.b = self.a + self.b, self.a
        if self.a < self.stop:
            return self.a
        raise StopIteration


fib = Fibonacci(100)
print(fib)

In [ ]:
for value in fib:
    print(value)

In [ ]:
print(list(Fibonacci(100)))

In [ ]:
fib = Fibonacci(100)
for i in range(70):
    print(i, next(fib))

Hence, an `iterator` is a `class` which has the methods `__iter__()` and `__next__()`.

A `generator` behaves very similar but does not require to write a `class`. The key component here is the `yield` command:

In [ ]:
def fibonacci(stop):
    """Generate Fibonacci numbers
    
    Arguments:
        stop (int): endpoint for the generation
    """
    a, b = 1, 1
    while True:
        a, b = a + b, a
        if a < stop:
            yield a
        else:
            break


print(fibonacci(100))

In [ ]:
for value in fibonacci(100):
    print(value)

In [ ]:
print(list(fibonacci(100)))

In [ ]:
fib = fibonacci(100)
for i in range(10):
    print(i, next(fib))